# 2023 데이터 크리에이터 캠프

## 전역 변수 설정

In [ ]:
datasetDir = os.path.join('D:\\', '2023dcc')
#datasetDir = '/root/dataset'

## MISSION 1. AI-Hub 에서 데이터셋 다운로드 및 학습데이터와 테스트 데이터셋 만들기

### 1-1. 첨부된 그림의 경로에서 TS7, TS8.ZIP는 Training VS2.ZIP는 Test 데이터셋으로 압축을 풀어 이미지 폴더를 만든다.
### Training 과 Test 데이터의 이미지 수는 각각 얼마인가?

In [ ]:
import zipfile
import os
from concurrent.futures import ThreadPoolExecutor

train_zip1_path = 'TS7.zip'
train_zip2_path = 'TS8.zip'
test_zip_path = 'VS2.zip'
train_folder = 'train'
test_folder = 'test'

if not os.path.exists(train_folder):
    os.makedirs(train_folder)

if not os.path.exists(test_folder):
    os.makedirs(test_folder)

def extract_zip(zip_path, target_folder): # 압축을 해제하는 함수
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(target_folder)

with ThreadPoolExecutor(max_workers=3) as executor: # 쓰레딩으로 동시 압축 해제
    executor.submit(extract_zip, train_zip1_path, train_folder)
    executor.submit(extract_zip, train_zip2_path, train_folder)
    executor.submit(extract_zip, test_zip_path, test_folder)

In [ ]:
import os

dataPaths = ['train', 'test']

for dataPath in dataPaths:
    folder_path = os.path.join(datasetDir, dataPath)
    image_extensions = ['.jpg', '.png', '.jpeg']
    
    image_count = 0
    
    for _, _, files in os.walk(folder_path):
        for file in files:
            if any(file.lower().endswith(ext) for ext in image_extensions):
                image_count += 1
    
    print(f"{folder_path} 폴더에 {image_count}개의 이미지가 있음.")

### 1-2. 이미지 이름에서 마지막 두자리가 01 : 마스크 미착용 02 : 마스크 부분 착용 03 : 마스크 착용을 의미한다.
### Training, Test에 각각 몇 개가 있는가?

In [ ]:
import os

valid_suffixes = ['01', '02', '03']

image_extensions = ['.jpg', '.jpeg', '.png']

dataPaths = ['train', 'test']

image_counts = {suffix: 0 for suffix in valid_suffixes} # 라벨이 키인 딕셔너리를 정의

for dataPath in dataPaths:
    folder_path = os.path.join(datasetDir, dataPath)
    for _, _, files in os.walk(folder_path):
        for file in files:
            if any(file.lower().endswith(ext) for ext in image_extensions):
                try: # 파일 이름에서 라벨을 불러와 딕셔너리에 카운트
                    index = file.lower().split('.')[0].split('_')[-1]
                    image_counts[index] += 1
                except: # 실패 시 파일명 반환
                    print(file)
                    exit(1)
            
for suffix, count in image_counts.items():
    print(f"'{suffix}'로 끝나는 파일은 {count}개 있음.")

## MISSION 2. 이미지 데이터 축소하기

### 2-1. 사진 크기가 너무 크기 때문에 사진을 축소해야 Colab에서 학습할 수 있을 것이다. 
### 어떤 크기로 이미지를 축소해야 하는지 결정하고, 실제 축소된 이미지 폴더를 만드시오.

In [ ]:
import os
import cv2

train_image_folder = 'train'
test_image_folder = 'test'

train_resized_folder = 'train_resize'
test_resized_folder = 'test_resize'

image_extensions = ['.jpg', '.png', '.jpeg']

target_size = (256, 256) # 이미지 사이즈

if not os.path.exists(train_resized_folder):
    os.makedirs(train_resized_folder)

if not os.path.exists(test_resized_folder):
    os.makedirs(test_resized_folder)

def resize_and_save_images(original_folder, resized_folder): # 원본 폴더에 있는 사진을 지정된 폴더에 리사이징하여 저장하는 함수
    for root, dirs, files in os.walk(original_folder):
        for filename in files:
            if any(filename.lower().endswith(ext) for ext in image_extensions):
                image_path = os.path.join(root, filename)
                relative_path = os.path.relpath(image_path, original_folder)
                output_path = os.path.join(resized_folder, relative_path) # 디렉토리 계산해서 저장 경로 지정
                os.makedirs(os.path.dirname(output_path), exist_ok=True)
                img = cv2.imread(image_path) # OpenCV로 이미지 읽어오기
                img = cv2.resize(img, target_size, interpolation=cv2.INTER_AREA) # OpenCV로 이미지 리사이징
                cv2.imwrite(output_path, img) # 이미지 저장

resize_and_save_images(train_image_folder, train_resized_folder)
resize_and_save_images(test_image_folder, test_resized_folder)

In [ ]:
import numpy as np
from PIL import Image
import os
from tqdm import tqdm

train_resized_folder = 'train_resize'
test_resized_folder = 'test_resize'

image_extensions = ['.jpg', '.png', '.jpeg']

target_size = (256, 256)

def load_images_with_labels_as_numpy_array(folder_path, target_size=(256, 256)): # 이미지를 읽어와서 넘파이 배열로 변환하는 함수
    image_data = []
    labels = []
    
    for root, dirs, files in os.walk(folder_path):
        for filename in tqdm(files):
            if any(filename.lower().endswith(ext) for ext in image_extensions):
                image_path = os.path.join(root, filename)
                img = Image.open(image_path)
                img = img.resize(target_size)
                img_array = np.array(img)
                
                index = filename.split('.')[0].split('_')[-1]
                
                image_data.append(img_array)
                labels.append(int(index))
    
    return np.array(image_data), np.array(labels) # 이미지 데이터와 라벨 데이터를 각각의 넘파이 배열로 저장

train_images, train_labels = load_images_with_labels_as_numpy_array(train_resized_folder)
test_images, test_labels = load_images_with_labels_as_numpy_array(test_resized_folder)

# 변환된 넘파이 배열을 각각의 npy 파일로 저장
np.save('train_images.npy', train_images)
np.save('train_labels.npy', train_labels)
np.save('test_images.npy', test_images)
np.save('test_labels.npy', test_labels)

### 2-2. 이미지 파일을 Colab으로 옮기는 것은 여러가지로 어렵다. 두가지 방식을 사용할 수 있다. 적절한 방법을 택해서 수행하시오.

(1) 로컬 컴퓨터에서 이미지를 넘파이 배열로 만들어 Colab으로 전송하는 방법 

(2) 이미지를 하나의 압축파일로 만들어 Colab으로 전송하는 방법

2-1에서 1번 방법을 사용하여, numpy 배열로 변환 후 파일로 저장.